In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite
/kaggle/input/amazon-fine-food-reviews/hashes.txt


In [2]:
#Loading datfrom CSV file to pandas dataframe

raw_data = pd.read_csv("../input/amazon-fine-food-reviews/Reviews.csv")

In [3]:
#Top 5 rows of the data

raw_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
#Number of rows and columns of the dataset

raw_data.shape

(568454, 10)

In [5]:
#Filtering out the data where score is not 3

raw_data = raw_data[raw_data['Score'] != 3]

In [6]:
raw_data.shape

(525814, 10)

In [7]:
#Since HelpfulnessNumerator cannot exceed HelpfulnessDenominator. All such reviews need to be removed. 

raw_data = raw_data[raw_data['HelpfulnessNumerator'] <= raw_data['HelpfulnessDenominator']]

In [8]:
raw_data.shape

(525812, 10)

In [9]:
#Function to mark reviews having score less than 3 as 0(negative) and score greater than 3 as 1(positive).

def pos_neg(x):
    if x < 3:
        return 0
    else:
        return 1

In [10]:
raw_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [11]:
#Applying the pos_neg function to the score column of the dataset

raw_data['Score'] = raw_data['Score'].apply(lambda z: pos_neg(z))

In [12]:
raw_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [13]:
#A particular user cannot review products more than once at a particular timestamp. Such rows need to be deleted

raw_data = raw_data.drop_duplicates(subset=("UserId","ProfileName","Time","Text"),keep = 'first',inplace = False)

In [14]:
raw_data.shape

(364171, 10)

In [15]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [16]:
#Set of all english stopwords

stop = set(stopwords.words('english'))

In [17]:
#Removing stopwords from all the reviews

for xyz in range(364171):
    raw_data['Text'].iloc[xyz] = ' '.join(x for x in raw_data['Text'].iloc[xyz].lower().split() if x not in stop)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
#Function to remove html tag

def html_tag_removal(data):
    bs = BeautifulSoup(data, "html.parser")
    stripped_text = bs.get_text(separator=" ")
    stripped_text = re.sub(r'[^A-Za-z0-9]+',' ',stripped_text)
    return stripped_text

#Function to remove tags

def tag_cleaner(data):
    data = data.replace('><'," ")
    data = data.replace('<',"")
    data = data.replace('>',"")
    return data

In [19]:
#Applying functions to the text column

raw_data['Text'] = raw_data['Text'].apply(lambda data: html_tag_removal(data))
raw_data['Text'] = raw_data['Text'].apply(lambda data: tag_cleaner(data))

In [20]:
#Generating TF-IDF vectors of the column Text

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=True)
train_vectors = vectorizer.fit_transform(raw_data["Text"])
feature_names = vectorizer.get_feature_names()

In [21]:
feature_names[:10]

['00', '000', '000mg', '001', '0099', '00am', '00pm', '01', '02', '03']

In [22]:
type(train_vectors)

scipy.sparse.csr.csr_matrix

In [23]:
train_vectors.shape

(364171, 33356)

In [24]:
#Generating training and testing data

xtrain = train_vectors[0:291336]
xtest = train_vectors[291336:]
ytrain = raw_data['Score'][0:291336]
ytest = raw_data['Score'][291336:]

In [25]:
xtrain.shape

(291336, 33356)

In [26]:
ytrain.shape

(291336,)

In [27]:
#Initializing Logistic Regression and training it.

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.2, max_iter=200)
model.fit(xtrain, ytrain)

LogisticRegression(C=0.2, max_iter=200)

In [28]:
#Predicting the label on xtest dataset and checking the ROC-AUC score

from sklearn.metrics import roc_auc_score
predictions = model.predict(xtest)
print('AUC: ', roc_auc_score(ytest, predictions))

AUC:  0.786529235222619
